In [1]:
import re
import nltk
import string
import warnings
import numpy as np
import pandas as pd
import gensim
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.stem.porter import * 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import BatchNormalization

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [2]:
df = pd.read_csv("./twitter_data.csv")
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [3]:
df['class'].value_counts()

2    24163
1    19190
0     1430
Name: class, dtype: int64

In [4]:
mentions = r'@[A-Za-z0-9]+'
hashes = r'#[A-Za-z0-9]+'
amps = r'&[A-Za-z0-9]+'
url_links = r'https?://[A-Za-z0-9./]+'
picture_url = r'pic.twitter.com/[A-Za-z0-9./]+'
www_link = r'www.[^ ]+'
comb = r'|'.join((mentions, url_links, picture_url, www_link, hashes, amps))
negations_dic = {"isn't":"isnot", "aren't":"arenot", "wasn't":"wasnot", "weren't":"werenot",
                "haven't":"havenot","hasn't":"hasnot","hadn't":"hadnot","won't":"willnot",
                "wouldn't":"wouldnot", "don't":"donot", "doesn't":"doesnot","didn't":"didnot",
                "can't":"cannot","couldn't":"couldnot","shouldn't":"shouldnot","mightn't":"mightnot",
                "mustn't":"mustnot",
                "is not":"isnot", "are not":"arenot", "was not":"wasnot", "were not":"werenot",
                "have not":"havenot","has not":"hasnot","had not":"hadnot","will not":"willnot",
                "would not":"wouldnot", "do not":"donot", "does not":"doesnot","did not":"didnot",
                "can not":"cannot","could not":"couldnot","should not":"shouldnot","might not":"mightnot",
                "must not":"mustnot"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):
    cleaned = re.sub(comb, '', text)
    try:
        decoded = cleaned.decode('utf-8-sig')
    except:
        decoded = cleaned
    
    lower_case = decoded.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub('[^a-zA-Z]', ' ', neg_handled)
    no_extra_white_space = re.sub('\s+', ' ', letters_only)
    return no_extra_white_space

In [5]:
clean_tweet_texts = []
for i in tqdm(range(0,len(df.index))):
    clean_tweet_texts.append(tweet_cleaner(str(df['tweet'][i])))

clean_df = pd.DataFrame(clean_tweet_texts,columns=['tweet'])
df['cleaned_tweet'] = clean_df.tweet

100%|█████████████████████████████████████████████████████████████████████████| 44783/44783 [00:01<00:00, 33764.08it/s]


In [6]:
df = df[['class','tweet','cleaned_tweet']]
df.head()

,class,tweet,cleaned_tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...,rt as a woman you shouldnot complain about cleaning up your house as a man you should always take the trash out
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!,rt boy dats cold tyga dwn bad for cuffin dat hoe in the st place
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit,rt dawg rt you ever fuck a bitch and she start to cry you be confused as shit
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny,rt g anderson based she look like a tranny
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;,rt the shit you hear about me might be true or it might be faker than the bitch who told it to ya


In [7]:
df['is_offensive'] = 0
df.loc[(df['class'] == 0),'is_offensive']=1
df.loc[(df['class'] == 1),'is_offensive']=1
df['is_offensive'].value_counts()

0    24163
1    20620
Name: is_offensive, dtype: int64

In [8]:
df = df.reset_index(drop=True)
df.head()

,class,tweet,cleaned_tweet,is_offensive
0,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...,rt as a woman you shouldnot complain about cleaning up your house as a man you should always take the trash out,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!,rt boy dats cold tyga dwn bad for cuffin dat hoe in the st place,1
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit,rt dawg rt you ever fuck a bitch and she start to cry you be confused as shit,1
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny,rt g anderson based she look like a tranny,1
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;,rt the shit you hear about me might be true or it might be faker than the bitch who told it to ya,1


In [9]:
df['cleaned_tweet'] = df['cleaned_tweet'].astype(str)
df.cleaned_tweet = df.cleaned_tweet.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
tokenized_tweet = df.cleaned_tweet.apply(lambda x: x.split())
tokenized_tweet.head()

0    [woman, you, shouldnot, complain, about, cleaning, your, house, man, you, should, always, take, the, trash, out]
1                                                [boy, dats, cold, tyga, dwn, bad, for, cuffin, dat, hoe, the, place]
2                                           [dawg, you, ever, fuck, bitch, and, she, start, cry, you, confused, shit]
3                                                                          [anderson, based, she, look, like, tranny]
4                               [the, shit, you, hear, about, might, true, might, faker, than, the, bitch, who, told]
Name: cleaned_tweet, dtype: object

In [10]:
stemmer = PorterStemmer() 
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) #stemming
tokenized_tweet.head()

0    [woman, you, shouldnot, complain, about, clean, your, hous, man, you, should, alway, take, the, trash, out]
1                                            [boy, dat, cold, tyga, dwn, bad, for, cuffin, dat, hoe, the, place]
2                                        [dawg, you, ever, fuck, bitch, and, she, start, cri, you, confus, shit]
3                                                                      [anderson, base, she, look, like, tranni]
4                          [the, shit, you, hear, about, might, true, might, faker, than, the, bitch, who, told]
Name: cleaned_tweet, dtype: object

In [11]:
for i in range(0,len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])    
df['cleaned_tweet'] = tokenized_tweet
df.head(10)

,class,tweet,cleaned_tweet,is_offensive
0,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...,woman you shouldnot complain about clean your hous man you should alway take the trash out,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!,boy dat cold tyga dwn bad for cuffin dat hoe the place,1
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit,dawg you ever fuck bitch and she start cri you confus shit,1
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny,anderson base she look like tranni,1
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;,the shit you hear about might true might faker than the bitch who told,1
5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;""",madison the shit just blow claim you faith and down for somebodi but still fuck with hoe,1
6,1,"!!!!!!""@__BrighterDays: I can not just sit up and HATE on another bitch .. I got too much shit going on!""",brighterday cannot just sit and hate anoth bitch got too much shit go,1
7,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of you big bitches coming for us skinny girls!!&#8221;,caus tire you big bitch come for skinni girl,1
8,1,""" &amp; you might not get ya bitch back &amp; thats that """,you mightnot get bitch back that that,1
9,1,""" @rhythmixx_ :hobbies include: fighting Mariam""\n\nbitch",hobbi includ fight mariam bitch,1


In [12]:
tokenized_tweet = df.cleaned_tweet.apply(lambda x: x.split())
tokenized_tweet.head(10)

0    [woman, you, shouldnot, complain, about, clean, your, hous, man, you, should, alway, take, the, trash, out]
1                                            [boy, dat, cold, tyga, dwn, bad, for, cuffin, dat, hoe, the, place]
2                                        [dawg, you, ever, fuck, bitch, and, she, start, cri, you, confus, shit]
3                                                                      [anderson, base, she, look, like, tranni]
4                          [the, shit, you, hear, about, might, true, might, faker, than, the, bitch, who, told]
5     [madison, the, shit, just, blow, claim, you, faith, and, down, for, somebodi, but, still, fuck, with, hoe]
6                            [brighterday, cannot, just, sit, and, hate, anoth, bitch, got, too, much, shit, go]
7                                                         [caus, tire, you, big, bitch, come, for, skinni, girl]
8                                                                  [you, mightnot, get, bitch, b

In [13]:
tfidf_vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1,3)) #unigram, bigram, trigram
tfidf_df = tfidf_vectorizer.fit_transform(df['cleaned_tweet'])
tfidf_df.shape

(44783, 500)

In [14]:
model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=500, # output dimensions
            window=5, # context window size
            min_count=5, # ignore any words that appear less than 5 times.                                  
            sg = 1, # skip-gram model
            hs = 0,
            negative = 10, # negative sampling
            workers= 12, # number of cores that can be used
) 

model_w2v.train(tokenized_tweet, total_examples= len(df['cleaned_tweet']), epochs=30)

(9664575, 12832650)

In [15]:
model_w2v.wv.most_similar(positive='bitch')

[('hoe', 0.5027706027030945),
 ('nomo', 0.4167632460594177),
 ('yassss', 0.40614134073257446),
 ('lyin', 0.3896762728691101),
 ('gbe', 0.3894902467727661),
 ('trippen', 0.38934704661369324),
 ('frontin', 0.38013243675231934),
 ('braceface', 0.3799997568130493),
 ('wallah', 0.378622829914093),
 ('lexi', 0.37748393416404724)]

In [16]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [17]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 500)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 500)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(44783, 500)

In [18]:
wordvec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.080567,0.087247,0.009612,-0.272802,0.011458,-0.061806,0.031311,0.092972,0.035659,-0.108299,...,0.036406,0.003328,0.093741,0.041980,-0.001982,-0.064888,-0.004784,0.042090,0.058333,-0.152933
1,-0.095384,0.111423,-0.018234,-0.343409,0.126451,-0.146284,-0.036135,0.057265,-0.034421,0.077683,...,0.046527,0.080112,-0.014584,0.102737,0.034078,0.022953,-0.085681,0.095434,0.037691,-0.041535
2,-0.068898,0.098352,0.136356,-0.313098,-0.005599,-0.091508,0.024152,0.003866,-0.078076,-0.050179,...,0.054432,0.039867,-0.005000,0.008658,-0.045559,0.007894,0.019463,0.028042,0.052773,-0.139427
3,0.044638,0.204748,0.104734,-0.171101,0.019810,-0.094981,0.043435,0.121041,0.051227,-0.108502,...,0.095133,-0.043309,0.039371,-0.041993,-0.167356,0.022635,-0.159386,0.174173,-0.049309,-0.161837
4,-0.131282,0.080979,0.100818,-0.346867,-0.047246,-0.053518,-0.016497,0.104746,-0.044078,-0.045439,...,0.095786,-0.005286,0.064299,0.049106,0.041619,0.024203,-0.105287,0.053482,0.080163,-0.153004


In [19]:
x = tfidf_df
y = df['is_offensive']

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [21]:
logistic_regression = LogisticRegression()

In [22]:
logistic_regression.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
y_pred = logistic_regression.predict(x_test)

In [24]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

95.82886745266167

In [25]:
f1_score(y_test, y_pred)

0.9538491945844452

In [26]:
x = wordvec_df
y = df['is_offensive']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [28]:
logistic_regression = LogisticRegression(max_iter=1000)

In [29]:
logistic_regression.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
y_pred = logistic_regression.predict(x_test)

In [31]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

94.85530546623794

In [32]:
f1_score(y_test, y_pred)

0.9436509489336724

In [33]:
embeddings_index = {}
for w in model_w2v.wv.vocab.keys():
    embeddings_index[w] = model_w2v.wv[w]

In [34]:
x = df['cleaned_tweet']
y = df['is_offensive']

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [36]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [37]:
for x in x_train[:5]:
    print (x)

dick call taco bell becaus bitch love their mouth but hate their ass
gucci mane jail and drop mixtap everi month and you hoe cannot even text back
hate when hoe caption shit who she lol shit hella cheesi
true but then again opinion hoe someon who goe and sleep with everybodi
afternoon all it far too nice outsid stuck the offic how everyon


In [38]:
sequences[:5]

[[182, 79, 1440, 1889, 144, 3, 22, 155, 580, 16, 106, 155, 34],
 [1485, 1207, 1066, 4, 523, 1799, 177, 487, 4, 2, 7, 70, 101, 318, 64],
 [106, 32, 7, 2314, 44, 65, 37, 29, 44, 896, 3244],
 [449, 16, 59, 146, 1486, 7, 221, 65, 460, 4, 160, 9, 425],
 [786, 14, 91, 491, 49, 112, 379, 801, 1, 662, 46, 176]]

In [39]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)

26

In [40]:
x_train_seq = pad_sequences(sequences, maxlen=30)
x_train_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  182,   79, 1440, 1889,  144,
           3,   22,  155,  580,   16,  106,  155,   34],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 1485, 1207, 1066,    4,  523, 1799,  177,
         487,    4,    2,    7,   70,  101,  318,   64],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,  106,   32,    7,
        2314,   44,   65,   37,   29,   44,  896, 3244],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  449,   16,   59,  146, 1486,
           7,  221,   65,  460,    4,  160,    9,  425],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,  786,   14,   91,  491,
          49,  112,  379,  801, 

In [41]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=30)

In [42]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 500))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [43]:
np.array_equal(embedding_matrix[3] ,embeddings_index.get('bitch'))

True

In [44]:
model_cnn = Sequential()
e = Embedding(100000, 500, weights=[embedding_matrix], input_length=30, trainable=False)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=150, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(200, activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.fit(x_train_seq, y_train, validation_data=(x_test_seq, y_test), epochs=5,  verbose=2)

Train on 33587 samples, validate on 11196 samples
Epoch 1/5
 - 7s - loss: 0.1227 - accuracy: 0.9586 - val_loss: 0.1051 - val_accuracy: 0.9646
Epoch 2/5
 - 7s - loss: 0.0845 - accuracy: 0.9727 - val_loss: 0.1214 - val_accuracy: 0.9571
Epoch 3/5
 - 7s - loss: 0.0655 - accuracy: 0.9783 - val_loss: 0.0979 - val_accuracy: 0.9718
Epoch 4/5
 - 7s - loss: 0.0477 - accuracy: 0.9838 - val_loss: 0.1096 - val_accuracy: 0.9670
Epoch 5/5
 - 7s - loss: 0.0366 - accuracy: 0.9880 - val_loss: 0.1410 - val_accuracy: 0.9690


In [45]:
y_pred = model_cnn.predict(x_test_seq)

In [46]:
for i in range(0,len(y_pred)):
    if (y_pred[i] >= 0.5) :
        y_pred[i] = 1
    else :
        y_pred[i] = 0

In [47]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

96.9006788138621

In [48]:
f1_score(y_test, y_pred)

0.9660768403558511

In [49]:
df.head()

,class,tweet,cleaned_tweet,is_offensive
0,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...,woman you shouldnot complain about clean your hous man you should alway take the trash out,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!,boy dat cold tyga dwn bad for cuffin dat hoe the place,1
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit,dawg you ever fuck bitch and she start cri you confus shit,1
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny,anderson base she look like tranni,1
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;,the shit you hear about might true might faker than the bitch who told,1


In [50]:
df['class'].value_counts()

2    24163
1    19190
0     1430
Name: class, dtype: int64

In [51]:
df['is_hate_speech'] = 0
df.loc[(df['class'] == 0),'is_hate_speech']=1

In [52]:
index = df[(df['class']==2)].index
dfo = df.copy()
dfo.drop(index, inplace = True)
df = dfo
df['class'].value_counts()

1    19190
0     1430
Name: class, dtype: int64

In [53]:
df['is_hate_speech'].value_counts()

0    19190
1     1430
Name: is_hate_speech, dtype: int64

In [54]:
df = df.reset_index(drop=True)
df.head()

,class,tweet,cleaned_tweet,is_offensive,is_hate_speech
0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!,boy dat cold tyga dwn bad for cuffin dat hoe the place,1,0
1,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit,dawg you ever fuck bitch and she start cri you confus shit,1,0
2,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny,anderson base she look like tranni,1,0
3,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;,the shit you hear about might true might faker than the bitch who told,1,0
4,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;""",madison the shit just blow claim you faith and down for somebodi but still fuck with hoe,1,0


In [55]:
tokenized_tweet = df.cleaned_tweet.apply(lambda x: x.split())
tokenized_tweet.head()

0                                           [boy, dat, cold, tyga, dwn, bad, for, cuffin, dat, hoe, the, place]
1                                       [dawg, you, ever, fuck, bitch, and, she, start, cri, you, confus, shit]
2                                                                     [anderson, base, she, look, like, tranni]
3                         [the, shit, you, hear, about, might, true, might, faker, than, the, bitch, who, told]
4    [madison, the, shit, just, blow, claim, you, faith, and, down, for, somebodi, but, still, fuck, with, hoe]
Name: cleaned_tweet, dtype: object

In [56]:
tfidf_vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1,3))
tfidf_df = tfidf_vectorizer.fit_transform(df['cleaned_tweet'])
tfidf_df.shape

(20620, 500)

In [57]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 500)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 500)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(20620, 500)

In [58]:
x = tfidf_df
y = df['is_hate_speech']

In [59]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [60]:
logistic_regression = LogisticRegression()

In [61]:
logistic_regression.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
y_pred = logistic_regression.predict(x_test)

In [63]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

93.50145489815714

In [64]:
precision_score(y_test, y_pred)

0.5930232558139535

In [65]:
recall_score(y_test, y_pred)

0.1452991452991453

In [66]:
f1_score(y_test, y_pred)

0.23340961098398172

In [67]:
x = wordvec_df
y = df['is_hate_speech']

In [68]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [69]:
logistic_regression = LogisticRegression(max_iter=1000)

In [70]:
logistic_regression.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
y_pred = logistic_regression.predict(x_test)

In [72]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

94.02521823472357

In [73]:
precision_score(y_test, y_pred)

0.5299145299145299

In [74]:
recall_score(y_test, y_pred)

0.19682539682539682

In [75]:
f1_score(y_test, y_pred)

0.28703703703703703

In [76]:
x = df['cleaned_tweet']
y = df['is_offensive']

In [77]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [78]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [79]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)

27

In [80]:
x_train_seq = pad_sequences(sequences, maxlen=30)

In [81]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=30)

In [82]:
model_cnn2 = Sequential()
e = Embedding(100000, 500, weights=[embedding_matrix], input_length=30, trainable=False)
model_cnn2.add(e)
model_cnn2.add(Conv1D(filters=150, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn2.add(GlobalMaxPooling1D())
model_cnn2.add(Dense(200, activation='relu'))
model_cnn2.add(BatchNormalization())
model_cnn2.add(Dense(1, activation='sigmoid'))
model_cnn2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn2.fit(x_train_seq, y_train, validation_data=(x_test_seq, y_test), epochs=5,  verbose=2)

Train on 15465 samples, validate on 5155 samples
Epoch 1/5
 - 3s - loss: 0.0903 - accuracy: 0.9785 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 2/5
 - 3s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 6.5491e-04 - val_accuracy: 1.0000
Epoch 3/5
 - 3s - loss: 4.5921e-04 - accuracy: 1.0000 - val_loss: 2.9900e-04 - val_accuracy: 1.0000
Epoch 4/5
 - 3s - loss: 2.2123e-04 - accuracy: 1.0000 - val_loss: 1.5901e-04 - val_accuracy: 1.0000
Epoch 5/5
 - 3s - loss: 1.2577e-04 - accuracy: 1.0000 - val_loss: 1.0510e-04 - val_accuracy: 1.0000


In [83]:
y_pred = model_cnn.predict(x_test_seq)

In [84]:
for i in range(0,len(y_pred)):
    if (y_pred[i] >= 0.5) :
        y_pred[i] = 1
    else :
        y_pred[i] = 0

In [85]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

47.02230843840931

In [86]:
precision_score(y_test, y_pred)

1.0

In [87]:
recall_score(y_test, y_pred)

0.4702230843840931

In [88]:
f1_score(y_test, y_pred)

0.6396622245678849